# HPC Minichallenge 1 - HS22

## Teil 1

### Aufgabe 1.4

In Kafka wird primär über vordefinierte Schnittstellen (Producer / Consumer) kommuniziert. Nutzer organisieren ihre Daten hierbei innerhalb von Topics. Kafka erlaubt es beim Erstellen neuer Topics die Anzahl an Partitionen als auch den Replikationsfaktor zu definieren. Der Replikationsfaktor bestimmt herbei, wie oft die Daten auf den verfügbaren Nodes dupliziert werden. Die Anzahl an Partitionen sind relevant, wenn es um die Parallelisierbarkeit der Schnittstellen geht.  Die genaue Verteilung der Daten in den Partitionen als auch die Aufteilung der Partitionen in einzelne Nodes organisiert Kafka eigenständig. Beim Monitoring via Kafdrop ist für jede Partition die Leader Node ersichtlich als auch die Priorität der Replikation Nodes. Das Abschalten einzelner Docker Container hat keinen grösseren Einfluss auf die Funktionsfähigkeit von Kafka, sofern der Replikationsfaktor genügend hoch gewählt ist. Solange mindestens eine Replikation in Kafka aktiv ist, ist es möglich weiterhin Informationen zu schreiben und abzugreifen. Der Kafka Zookeeper vergibt automatisch die ausgefallenen Leader auf die verfügbaren Replikation Nodes. Werden nun Partitionen auf Nodes durch das Wiedereinschalten von Containern erneut verfügbar, bleiben diese scheinbar als Leader Nodes disqualifiziert. 

### Aufgabe 1.7


#### Daten: 
Die live Bewertungen werden durch den bestehenden Datensatz **Toys_and_Games_5.json** simuliert. Dieses ist einem Subset des Amazon Review Dataset von Ni Jianmo welches unter dem folgenden URL erreichbar ist: 
https://nijianmo.github.io/amazon/index.html

#### Scenario: 
Ein Onlinehändler möchte auf seiner Webseite neu ein Live-Feed implementieren, mit dem Ziel, die Kunden dazu zu ermutigen mehr Produkte anzusehen. Dafür sollen im Live-Feed jeweils die fünf trendigen Produkte der letzten Minute mit den aktuellsten Kundenratings angezeigt werden. 
Für die Umsetzung möchte er die bereits bestehende Datenmanagement-Infrastruktur nutzen, in dem Ratings mittels Apache Kafka strukturiert werden. 
Diese ist in *Abbildung 1* aufgeführt.

![Schema_1.png](Schema_1.png)

#### Struktur:

Das Kafka Environment ist auf verschiedene Images verteilt. Dieses beinhaltet das Kafka Cluster bestehend aus den drei Broker images *broker1*, *broker2* und *broker3*, als auch dem Zookeper *zookeper*, welcher die Kommunikation und Datenstrukturierung unter den Broker organisiert und in einem separaten Image läuft. Als zusätzlicher Mikroservice fürs Monitoring steht, das Kafdrop web UI zur Verfügung, welches über ein weiteres image *kafdrop1* ausgeführt wird.

Im *jupyter1* Container wird sämtlicher in Python geschriebener Code ausgeführt.
Hier befinden sich auch die vier Notebooks, von welchen der gesammte Prozess gesteuert werden kann.

- **Producer_1.ipynb** simuliert die live Bewertungen der Kunden. Hierfür werden mit einer definierbaren Frequenz > 1 Hz einzelne Bewertungen aus dem Toys_and_Games_5.json gelesen und zu Kafka unter dem Topic `live_ratings` gesendet. 

- **Consumer_1.ipynb** erhält alle live Bewertungen, welche auf Kafka unter dem Topic  `live_ratings` publiziert wurden. In Kombination mit einem Datasink werden in minütlichen Abständen die Anzahl an Bewertungen pro Produkt als auch die durchschnittliche Produktbewertung errechnet und abgespeichert.

- **Producer_2.ipynb** überprüft, ob neue Durchschnittsbewertungen vorhanden sind und publiziert jeweils die fünf Produkte mit den häufigsten Bewertungen auf ein neues Kafka Topic `top_ratings`.

- **Consumer_2.ipynb** erhält die fünf  meist bewerteten Produkte aus dem Topic `top_ratings` und zeigt diese live als Barplot an.

#### Verbesserungespotential

Zukünftig können die einzelnen Scripts auf mehrere Container verteilt werden. Dadurch wäre es möglich, diese auf unterschiedliche Systeme laufen zu lassen.

### 1. Bonusaufgabe:

To do...

## Teil 2

### Aufgabe 2.1

In diesem Schritt wird RabbitMQ als weiterer Message Broker hinzugefügt. 

Dafür wurde in einem ersten Schritt das **docker-compose.yml** angepasst. Hier wurde ein neuer Container *rabbitmq1* erstellt, welcher das rabbitmq:3.8-management-alpine image von docker-hub herunterlädt. Der Zusatz "Management" bedeutet hier, dass wir über einen zusätzlichen Port automatisch ein UI mitgeliefert erhalten, welches zum Monitoren verwendet werden kann. Dieses ist via [localhost:15672](localhost:15672) erreichbar

"Alpine" bedeute hingegen legidlich, dass es sich um die speichereffiziente Variante handelt.

Weiter wurden die bestehenden Klassen und ihre Methoden im **helper_file.py** angepasst und erweitert, sodass nun einfach zwischen Kafka und RabbitMQ gewechselt werden kann.

### Aufgabe 2.2

Nachfolgend in Visualisierung 2 ist nun die Implementation des Message Brokers mittels RabbitMQ visualisiert. 

![Schema_2.png](Schema_2.png)


Anders als bei der Implementation mit Kafka wird hier aktuell nur ein Container genutzt, über welchen das ganze System läuft. Es ist jedoch auch mit RabbitMQ möglich ein Cluster mit mehreren Containern zu erstellen. Der *jupyter1* Container bleibt im Vergleich zur vorherigen Implementation unverändert. 

### Aufgabe 2.3

RabbitMQ kommuniziert standardmässig mit dem binären AMQP-0.9.1 Protokoll, wobei auch noch andere Protokolle unterstützt werden. Siehe [www.rabbitmq.com/protocols](https://www.rabbitmq.com/protocols.html#:~:text=AMQP%200%2D9%2D1%20is%20the,protocol%20used%20by%20RabbitMQ%20tutorials) 

RabbitMQ selbst funktioniert als pub-sub Kommunikationspattern. Die Daten werden beim publishen mit RabbitMQ zuerst auf eine Exchange gesendet, wobei es mehrere Arten von Exchanges gibt. Ich nutze den Exchange-Typ `Direkt`, was bedeutet, dass unter Verwendung eines Message-Routing-Schlüssel Nachrichten von der Exchange kopiert und die Kopien direkt zu einer Queue hinzugefügt werden.

Alternative Exchange-Typen wären:
- `Topic`: Hier werden Nachrichten basierend auf sub-strings einer oder auch mehreren Queues zugeordnet. Dabei muss die Queue gleich heissen wie der relevante Sub-string.

- `Fanout`: Verteilt alle Nachrichten an alle an eine Exchange gebundene Exchange. Der Message-Routing-Schlüssel wird hier ignoriert.

- `Header`: Gleich wie Topic, aber anstelle des Message-Routing-Schlüssels werden hier Informationen aus dem Header der Nachricht fürs Routing verwendet. 

Anschliessend können die Queues über den passenden Message-Routing-Schlüssel wieder abgerufen werden. Wird vom Nutzer keine Exchange definiert, wird standardmässig eine immer vorhandene Standard-Exchange `""` verwendet.

#### Kafka vs RabbitMQ
Da RabbitMQ mit Queues arbeitet, werden Nachrichten, nachdem sie konsumiert wurden und eine Bestätigung zum Konsum gesendet wurde, auch tatsächlich gelöscht. Kafka hingegen kann auch als Queue genutzt werden, die dahinterliegende Speicherarchitektur ist jedoch deutlich flexibler und die Lebensdauer von Daten in Kafka wird durch eine Policy festgelegt. Während Kafka mit dem Zookeeper einen grossen Teil der Speicherverwaltung selbst übernimmt, kann bei RabbitMQ das exakte Datenrouting in der Applikation festgelegt werden, als auch Prioritäten im Datenfluss definiert werden.



#### Weitere Kommunikationsarten: 
*System-to-system*: Hierbei handelt es sich um einen traditionellen Ansatz, in dem kein weiterer Message-Broker dazwischen fungiert. Zwar bietet diese Methode für kleine Systeme mit nur sehr wenigen Akteuren an. Sie ist jedoch erstens relativ langsam, da der Sender nicht mehr Daten produzieren sollte als Empfänger verarbeiten kann, da sonst Datenverlust droht. Und zweitens ist diese Variante nur schlecht skalierbar, da für jeden weiteren Akteur im Netz eigene Schnittstellen programmiert werden müssen, was schnell exponentiell Skaliert.

#### Praktikabilität des Kommunikationspattern 
Ich bin der Meinung, dass das gewöhnliche pup-sub Kommunikationspattern mit RabbitMQ nur teilweise geeignet ist, da die Ratings nach dem Konsum gelöscht werden und vom Webshop ein Interesse bestehen sollte, Produktbewertungen Persistent aufzubewahren. In diesem Falle wäre also die Verwendung von Kafka als Message-Broker definitiv sinnvoller. 